# 실습 09: NER 기반 핵심 정보 추출

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab09_NER정보추출.ipynb)

## 🎯 학습 목표
- NER(Named Entity Recognition) 이해
- 규칙 기반 정보 추출
- 실전 문서에서 핵심 정보 자동 추출

## ⏱️ 소요 시간: 30분
## 📊 난이도: ⭐⭐⭐☆☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git paddlepaddle paddleocr
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# 계약서 샘플 이미지 생성
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 한글 폰트 로드
try:
    font_title = ImageFont.truetype("C:\\Windows\\Fonts\\malgunbd.ttf", 32)
    font_label = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 16)
    font_data = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 18)
except:
    try:
        font_title = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf", 32)
        font_label = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 16)
        font_data = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 18)
    except:
        font_title = ImageFont.load_default()
        font_label = ImageFont.load_default()
        font_data = ImageFont.load_default()

# 계약서 이미지 생성
img = Image.new('RGB', (800, 600), color='white')
draw = ImageDraw.Draw(img)

# 제목
draw.text((300, 30), "용역 계약서", fill='black', font=font_title)

# 계약 정보
y_pos = 100
contract_data = [
    ("계약일자:", "2025년 3월 1일"),
    ("발주자:", "삼성전자 주식회사"),
    ("담당자:", "김철수 부장"),
    ("연락처:", "02-2000-1234"),
    ("이메일:", "chulsoo.kim@samsung.com"),
    ("", ""),
    ("수주자:", "(주)테크솔루션"),
    ("연락처:", "031-456-7890"),
    ("이메일:", "contact@techsolution.com"),
    ("", ""),
    ("계약금액:", "50,000,000원"),
    ("입금계좌:", "123-456-789012"),
]

for label, value in contract_data:
    if label:
        draw.text((50, y_pos), label, fill='gray', font=font_label)
        draw.text((200, y_pos), value, fill='black', font=font_data)
    y_pos += 35

img.save('contract_doc.jpg')
print("✅ 계약서 이미지 생성 완료!")
img


In [ ]:
# PaddleOCR로 텍스트와 위치 정보 추출
from paddleocr import PaddleOCR
import cv2

print("📄 OCR 실행 중...")
ocr = PaddleOCR(use_textline_orientation=True, lang='korean')
result = ocr.predict('contract_doc.jpg')

# OCR 결과 추출
ocr_result = result[0]
texts = ocr_result['rec_texts']
scores = ocr_result['rec_scores']
boxes = ocr_result['dt_polys']

print(f"✅ OCR 완료: {len(texts)}개의 텍스트 영역 검출\n")
print("="*70)
print("검출된 텍스트:")
print("="*70)
for i, (text, score) in enumerate(zip(texts, scores), 1):
    print(f"{i:2d}. {text:30s} (신뢰도: {score:.2%})")


In [ ]:
# 이미지에 NER 결과 시각화 (엔티티 타입별 색상 표시)
import matplotlib.pyplot as plt
import cv2
import numpy as np

# 엔티티 타입별 색상 정의
entity_colors = {
    'DATE': (255, 100, 100),       # 빨강
    'ORG': (100, 255, 100),        # 초록
    'PERSON': (100, 100, 255),     # 파랑
    'PHONE': (255, 255, 100),      # 노랑
    'EMAIL': (255, 100, 255),      # 마젠타
    'MONEY': (100, 255, 255),      # 시안
    'ACCOUNT': (200, 150, 100),    # 갈색
    'OTHER': (150, 150, 150),      # 회색
}

# 이미지 로드
img = cv2.imread('contract_doc.jpg')
img_annotated = img.copy()

# 한글 폰트 로드 (레이블용)
try:
    from PIL import Image, ImageDraw, ImageFont
    label_font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 14)
except:
    try:
        label_font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 14)
    except:
        label_font = ImageFont.load_default()

# PIL로 변환 (한글 표시용)
img_pil = Image.fromarray(cv2.cvtColor(img_annotated, cv2.COLOR_BGR2RGB))
draw = ImageDraw.Draw(img_pil)

# 각 텍스트 영역에 바운딩 박스와 라벨 표시
for item in ocr_with_ner:
    bbox = item['bbox']
    entity_type = item['entity_type']
    text = item['text']
    
    # 색상 선택
    color = entity_colors.get(entity_type, (150, 150, 150))
    
    # 바운딩 박스 좌표
    points = np.array(bbox, dtype=np.int32)
    x_min, y_min = points[:, 0].min(), points[:, 1].min()
    x_max, y_max = points[:, 0].max(), points[:, 1].max()
    
    # OpenCV로 박스 그리기
    cv2.polylines(img_annotated, [points], True, color, 2)
    
    # PIL로 라벨 텍스트 그리기
    label = f"{entity_type}"
    draw.rectangle([x_min-2, y_min-22, x_min+len(label)*10, y_min-2], fill=color)
    draw.text((x_min, y_min-20), label, fill='white', font=label_font)

# PIL을 다시 OpenCV로 변환
img_annotated = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# 원본 이미지
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('원본 이미지', fontsize=14, fontproperties=fontprop, pad=15)
axes[0].axis('off')

# NER 결과 표시된 이미지
axes[1].imshow(cv2.cvtColor(img_annotated, cv2.COLOR_BGR2RGB))
axes[1].set_title('NER 결과 매핑 (엔티티 타입별 색상)', fontsize=14, fontproperties=fontprop, pad=15)
axes[1].axis('off')

plt.tight_layout()
plt.show()

# 범례 표시
print("\n" + "="*70)
print("🎨 엔티티 타입별 색상")
print("="*70)
for entity_type, color in entity_colors.items():
    if any(item['entity_type'] == entity_type for item in ocr_with_ner):
        rgb = f"RGB{color}"
        print(f"  {entity_type:12s} : {rgb}")

print("\n✅ NER 매핑 시각화 완료!")


In [ ]:
# 구조화된 데이터로 변환 및 저장
import pandas as pd

# 엔티티 타입별로 그룹화
data_by_type = {}
for item in ocr_with_ner:
    entity_type = item['entity_type']
    if entity_type != 'OTHER':
        if entity_type not in data_by_type:
            data_by_type[entity_type] = []
        data_by_type[entity_type].append(item['text'])

print("\n" + "="*70)
print("📋 추출된 정보 요약")
print("="*70)
for entity_type, values in data_by_type.items():
    print(f"\n[{entity_type}]")
    for value in values:
        print(f"  • {value}")

# DataFrame으로 저장
data = []
for item in ocr_with_ner:
    if item['entity_type'] != 'OTHER':
        data.append({
            '엔티티 타입': item['entity_type'],
            '텍스트': item['text'],
            'OCR 신뢰도': f"{item['ocr_score']:.2%}",
            'NER 신뢰도': f"{item['entity_conf']:.2%}"
        })

df = pd.DataFrame(data)
print("\n" + "="*70)
print("📊 전체 추출 결과")
print("="*70)
print(df.to_string(index=False))

# CSV로 저장
df.to_csv('contract_entities.csv', index=False, encoding='utf-8-sig')
print("\n✅ CSV 저장 완료: contract_entities.csv")


In [ ]:
# NER 시스템으로 각 텍스트 영역 분류
from docai_course.ner import UnifiedNERSystem

ner = UnifiedNERSystem()

# 각 텍스트에 NER 적용
ocr_with_ner = []
for text, score, bbox in zip(texts, scores, boxes):
    # NER 분석
    entities = ner.rule_based_ner(text)
    
    # 가장 신뢰도 높은 엔티티 선택
    if entities:
        entity_type = entities[0]['entity']
        entity_conf = entities[0]['confidence']
    else:
        entity_type = 'OTHER'
        entity_conf = 0.0
    
    ocr_with_ner.append({
        'text': text,
        'bbox': bbox,
        'ocr_score': score,
        'entity_type': entity_type,
        'entity_conf': entity_conf
    })

print("\n" + "="*70)
print("📊 NER 분류 결과")
print("="*70)
for i, item in enumerate(ocr_with_ner, 1):
    print(f"{i:2d}. [{item['entity_type']:15s}] {item['text']:30s} (NER신뢰도: {item['entity_conf']:.2%})")
